<a href="https://colab.research.google.com/github/tensorflow-project/FineTuning/blob/main/image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/tensorflow-project/FineTuning

fatal: destination path 'FineTuning' already exists and is not an empty directory.


In [2]:
import sys
import os
import shutil
import numpy as np
from google.colab import drive
from PIL import Image
import textual_inversion as txt
import shutil

py_file_location = "/content/FineTuning"
sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/FineTuning/models"
sys.path.append(os.path.abspath(py_file_location))

ModuleNotFoundError: ignored

In [ ]:
stable_diffusion = txt.stable_diffusion
placeholder_token = txt.placeholder_token

In [ ]:
### choose where to load the weights from, either from your google drive or you load our pretrained weights
### make sure to insert the exact name of your weight.npy
drive.mount("/content/drive")
path = '/content/drive/MyDrive/2_weights10.npy'

In [ ]:
###load the array of the weights of the text encoder from the training
text_encoder_weights = np.load(path, allow_pickle=True)

### Set the weights of the text encoder
stable_diffusion.text_encoder.set_weights(text_encoder_weights)

In [ ]:
### insert your prompt here
prompt = f"a angry {placeholder_token}."
number = 2

In [ ]:
### generate images and save them directly in your Google Drive


### where to store your images, you may have to create the folder "Images" first
drive_folder = '/content/drive/MyDrive/bildeeer/'

txt.image_generation(prompt, drive_folder, number)

In [ ]:
### generate images and plot them here
### choose number of images to be generated by changing batch_size
### change seed if you want different images with same weights
generated = stable_diffusion.text_to_image(
    prompt , batch_size=3,  num_steps=30, seed=2332
)
txt.plot_images(generated)